In [1]:
import sys
!{sys.executable} -m pip  install --quiet pyarrow memray pandas watermark ipywidgets bs4 polars rich humanize

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


Inspired by [Peter's notebook here](https://colab.research.google.com/drive/1TWa9L5NQE-cBYOpFvzwSu7Rjoy6QtFWw#scrollTo=OTEolvIk4Cmb)

In [25]:
from tempfile import NamedTemporaryFile
from pathlib import Path
import os

import pandas as pd
import humanize

from bs4 import BeautifulSoup
from rich import print

In [13]:
%load_ext memray

In [16]:
# Use arrow directly: https://arrow.apache.org/docs/python/json.html
# sample with one row of data
table = json.read_json("/Users/vicki/viberary/viberary/jsonparquet/src/main/resources/goodreads_sample.json")

In [17]:
table 

pyarrow.Table
isbn: string
text_reviews_count: string
series: list<item: null>
  child 0, item: null
country_code: string
language_code: string
popular_shelves: list<item: struct<count: string, name: string>>
  child 0, item: struct<count: string, name: string>
      child 0, count: string
      child 1, name: string
asin: string
is_ebook: string
average_rating: string
kindle_asin: string
similar_books: list<item: string>
  child 0, item: string
description: string
format: string
link: string
authors: list<item: struct<author_id: string, role: string>>
  child 0, item: struct<author_id: string, role: string>
      child 0, author_id: string
      child 1, role: string
publisher: string
num_pages: string
publication_day: string
isbn13: string
publication_month: string
edition_information: string
publication_year: string
url: string
image_url: string
book_id: string
ratings_count: string
work_id: string
title: string
title_without_series: string
----
isbn: [["0743509986"]]
text_reviews_c

In [27]:
%%memray_flamegraph
df = table.to_pandas()

⠋ Processing allocation records... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

Results saved to memray-results/tmpmgawbhfs/flamegraph.html

In [28]:
df

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,0743509986,6,[],US,,"[{'count': '2634', 'name': 'to-read'}, {'count...",,false,3.23,B000FC0PBC,...,10,Abridged,2001,https://www.goodreads.com/book/show/1333909.Go...,https://s.gr-assets.com/assets/nophoto/book/11...,1333909,10,1323437,Good Harbor,Good Harbor


In [29]:
# total memory allocated for DF
latest_file = max(Path("memray-results/").glob("**/*.html"), key=os.path.getmtime)
soup = BeautifulSoup(latest_file.read_text())
stats = soup.find("div", {"id" : "statsModal"}).find("div", {"class" : "modal-body"})
print(stats.text)

Command line: /usr/local/lib/python3.9/site-packages/ipykernel_launcher.py -f 
/Users/vicki/Library/Jupyter/runtime/kernel-87e391e0-0452-4dc9-bbd9-48ccec32f39e.json
          Start time: 2023-01-11 13:15:56.652000
          End time: 2023-01-11 13:15:56.705000
          Total number of allocations: 974
          Total number of frames seen: 78
          Peak memory usage: 19.3 kB
          Python allocator: pymalloc

In [ ]:
import polars as pl
df = pl.read_json("/Users/vicki/viberary/viberary/jsonparquet/src/main/resources/goodreads_data.json")

In [ ]:
# https://stackoverflow.com/questions/73154392/read-json-on-polars-causes-outofspec-error